# <h1 align="center">Cow Sprinker Optimization </h1>

## Information

### Background:

In dairy farms cows are relieved during hot days by a system of sprayed water. Diamond J Dairy Farmoperates a system of equally spaced spouts on one side of the fence that all turn on at the same time and spray the respective adjacent area, which is abouta cow width. Cows move into the sprayed area to be relieved when the water is on. Conserving water is important to all farmers. Soaker lines above the feed area on modern dairies is a primary way to cool animals. The soaker line is automated at a central spot and is activated based on outside temperature. When the weather triggers water line to be activated,the soaker linewets groups of 140-160 cows at a time for 1 minute on and 4-10 minutes off.

### Problem(s):


When the water turns on, not all soakers havea cow in the respective soaking area, resulting in significant water waste. When there is no animal in reach of the soaker the water could be conserved.

### Objectives:

The final goal is a system that would detect if a cow was in position when the water soaker was set to come on. If the cow isnot in the area of reach of the water, then the water shouldnot come on for that space, however the water shouldcome on in other areas in the same water line where there are cows.The first step is to design a vision-basedsystem whereby one camera is positioned to view multiple soaker areas, and software that detects whether or not there is a cow in each of them. The software will later be used to activate each soaker valve.

## General Imports

In [1]:
#!pip install boto3

In [1]:
#Import Numpy
import numpy as np

#Importing Pillow and its Functions
import PIL
from PIL import ImageEnhance 
from PIL import Image as im

#Importing Open CV2 
import cv2 as cv
import os
from time import time

#Importing tkinter
from tkinter import *
import random

#Max_Int
import sys

#boto3
import boto3

ACCESS_ID = 0
ACCESS_KEY = 0
DEFAULT_REGION = 'us-west-2'

s3 = boto3.resource('s3',
         aws_access_key_id=ACCESS_ID,
         aws_secret_access_key= ACCESS_KEY)

## Helper Functions

### File to png

In [2]:
def FiletoPng(file):
    imgJPG = Image.open(file)
    parts = file.split(".")
    pngfile = parts[0] + "-" + ".png" # string 
    imgJPG.save(pngfile)
    return pngfile

### Arraytojpg

In [15]:
def arraytojpg(frame,frame_counter):
    folder_name = base = "Frames"

    # create folder for images in current path if not exists
    current_path = os.getcwd()
    folder_path = os.path.join(current_path, folder_name)
    
    #file_path = os.path.join(folder_path, str(frame_counter) + "cows.jpg")
    file_path = os.path.join("cows.jpg")
    cv.imwrite(file_path, frame)

### Set up

In [4]:
def setup(boundingbox_widget):
    print("Instructions:")
    print("Mark the stanchions with Xs")
    print("Press Q to Exit Window")
    x = True
    while x:
        cv2.imshow('image', boundingbox_widget.show_image())
        key = cv2.waitKey(1)

        # Close program with keyboard 'q'
        if key == ord('q'):
            cv2.destroyAllWindows()
            x = False
            exit(1)

In [5]:
import cv2

image_coor = []

class BoundingBoxWidget(object):
    
    def __init__(self,frame):
        
        self.original_image = frame
        self.clone = self.original_image.copy()

        cv2.namedWindow('image')
        cv2.setMouseCallback('image', self.extract_coordinates)

        # Bounding box reference points
        self.image_coordinates = []
        
    def extract_coordinates(self, event, x, y, flags, parameters):
        # Record starting (x,y) coordinates on left mouse button click
        if event == cv2.EVENT_LBUTTONDOWN:
            self.image_coordinates = [(x,y)]

        # Record ending (x,y) coordintes on left mouse button release
        elif event == cv2.EVENT_LBUTTONUP:
            self.image_coordinates.append((x,y))
            print('top left: {}, bottom right: {}'.format(self.image_coordinates[0], self.image_coordinates[1]))
            
            image_coor.append([self.image_coordinates[0],self.image_coordinates[1]])
            
            # Draw rectangle 
            cv2.line(self.clone, self.image_coordinates[0], self.image_coordinates[1], (36,255,12), 2)
            cv2.imshow("image", self.clone) 

        # Clear drawing boxes on right mouse button click
        elif event == cv2.EVENT_RBUTTONDOWN:
            self.clone = self.original_image.copy()

    def show_image(self):
        return self.clone
   

### Video to Frames

In [6]:
def VideoInfoExtractor(video,seconds_frequency):
    
    cap = cv.VideoCapture(video)
    Num_frame = int(cap.get(cv.CAP_PROP_FRAME_COUNT))
    FPS = cap.get(cv.CAP_PROP_FPS) # Gets the frames per second
    mul = FPS * seconds_frequency # calculates number of frames that creates 10 seconds of video
    
    # Check if camera opened successfully
    if (cap.isOpened()== False):
        print("Error opening video")

    return cap,Num_frame,FPS,mul

In [7]:
def VideotoFrames(cap,frame_counter):
    
    cap.set(cv.CAP_PROP_POS_FRAMES, frame_counter)
    ret, frame = cap.read()
    
    return frame

### RunLiveVideo

In [8]:
#for streaming
def RunLiveVideo(videoUrl):
    video = cv2.VideoCapture(videoUrl)
    
    while True:
        _,frame = video.read()
        cv2.imshow("RTSP", frame)
        k = cv2.waitKey(1)
        if k == ord('q'):
            break
    
    video.release()
    cv2.destroyAllWindows()

### Image crop

In [9]:
def imageCrop(image):
    cropped_image = image[0:600, 0:940]
    return cropped_image

### AWS

In [10]:
def Matrix(lI,tI,image_coor):
    
    xCoor = np.zeros(1000)
    yCoor = np.zeros(1000)
    finalMatrix = np.zeros(20)

    counter = 0

    for X in image_coor:
        for line in X:
            for coordinate in line:
                if(line.index(coordinate) != 1):
                    xCoor[counter] = coordinate
                    counter = counter + 1
                else:
                    yCoor[counter] = coordinate
                    counter = counter + 1

    counter = 0

    for data in lI:
        if(counter != len(lI)):
            if(data >= xCoor[counter]):
                if (data <= xCoor[counter + 1]):
                    finalMatrix[counter] = 1
                    counter = counter + 1
                else:
                    finalMatrix[counter] = 0
                    counter = counter + 1
            else:
                finalMatrix[counter] = 0
                counter = counter + 1

    counter = 0

    for data in tI:
        if(counter != len(tI)):
            if(data >= yCoor[counter]):
                if (data <= yCoor[counter + 1]):
                    finalMatrix[counter] = 1
                    counter = counter + 1
                else:
                    finalMatrix[counter] = 0
                    counter = counter + 1
            else:
                finalMatrix[counter] = 0
                counter = counter + 1
    return finalMatrix

In [19]:
def CallAWS(frame,frame_counter):
    arraytojpg(frame,frame_counter)
    global s3
    
    ## Adding local jpg to S3 to be called by model ##
    s3 = boto3.client('s3')
    bucket = 'cowstoscan'
    key_name = 'cows.jpg'
    s3.upload_file('cows.jpg', bucket, key_name)

    ## Creating client instance ##
    client = boto3.client('rekognition')
    
    ## Calling Model ##
    response    = client.detect_custom_labels(ProjectVersionArn = 'arn:aws:rekognition:us-west-2:551144275003:project/CowModel/version/CowModel.2021-11-29T16.38.24/1638232704374', Image = {'S3Object': {'Bucket': 'cowstoscan', 'Name': 'cows.jpg' } })
    
    ## Deleting object from S3 now that model has been called ##
    s3.delete_object(Bucket = 'cowstoscan', Key = 'cows.jpg')
    s3.Object('cowstoscan', 'cows.jpg').delete()
    
    if os.path.exists("cows.jpg"):
          os.remove("cows.jpg")
            
    #response = "{'CustomLabels': [{'Name': 'Cow', 'Confidence': 98.71299743652344, 'Geometry': {'BoundingBox': {'left': 0.04439999908208847, 'Height': 0.08868999779224396, 'Left': 0.5378699898719788, 'Top': 0.20231999456882477}}}, {'Name': 'Cow', 'Confidence': 96.47700500488281, 'Geometry': {'BoundingBox': {'left': 0.03897999972105026, 'Height': 0.13030999898910522, 'Left': 0.22544999420642853, 'Top': 0.3942900002002716}}}, {'Name': 'Cow', 'Confidence': 96.3479995727539, 'Geometry': {'BoundingBox': {'left': 0.04901000112295151, 'Height': 0.18794000148773193, 'Left': 0.25672999024391174, 'Top': 0.36059001088142395}}}, {'Name': 'Cow', 'Confidence': 95.70700073242188, 'Geometry': {'BoundingBox': {'left': 0.04177999868988991, 'Height': 0.0692799985408783, 'Left': 0.6096100211143494, 'Top': 0.15031999349594116}}}, {'Name': 'Cow', 'Confidence': 92.4020004272461, 'Geometry': {'BoundingBox': {'left': 0.06054000183939934, 'Height': 0.1060900017619133, 'Left': 0.49428001046180725, 'Top': 0.2153100073337555}}}, {'Name': 'Cow', 'Confidence': 92.11599731445312, 'Geometry': {'BoundingBox': {'left': 0.08161000162363052, 'Height': 0.2087700068950653, 'Left': 0.001180000021122396, 'Top': 0.6182699799537659}}}, {'Name': 'Cow', 'Confidence': 86.81700134277344, 'Geometry': {'BoundingBox': {'left': 0.06851000338792801, 'Height': 0.18070000410079956, 'Left': 0.318589985370636, 'Top': 0.30358999967575073}}}, {'Name': 'Cow', 'Confidence': 84.68299865722656, 'Geometry': {'BoundingBox': {'left': 0.07779999822378159, 'Height': 0.11180999875068665, 'Left': 0.41760000586509705, 'Top': 0.2549299895763397}}}], 'ResponseMetadata': {'RequestId': '86a4a7e2-69ee-472f-9495-be3cd4ad74e0', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '86a4a7e2-69ee-472f-9495-be3cd4ad74e0', 'content-type': 'application/x-amz-json-1.1', 'content-length': '1486', 'date': 'Sat, 04 Dec 2021 03:27:06 GMT'}, 'RetryAttempts': 0}}"
    responseStr = str(response)

    nameConf  = "'Name': 'Cow', 'Confidence':"
    leftData = "'Left': "
    topData   = "'Top': "

    leftInfo = []
    topInfo   = []

    num = 1

    #Finding amount of cows
    for i, _ in enumerate(responseStr):
        if responseStr[i:i + len(nameConf)] == nameConf:
            print('Cow ' + str(num) + ' at index ' + str(i))
            num += 1

    for i, _ in enumerate(responseStr):
        if responseStr[i:i + len(leftData)] == leftData:
            leftInfo.append(float(responseStr[i + 8:i + 25]))

    for i, _ in enumerate(responseStr):
        if responseStr[i:i + len(topData)] == topData:
            topInfo.append(float(responseStr[i + 7:i + 25]))

    lI = np.asarray(leftInfo)
    tI = np.asarray(topInfo)

  
    lI = (lI * 1920).astype(int)
    tI = (tI * 1080).astype(int)
    
    #print("left Info ")
    #print(lI)

    #print("Top Info")
    #print(tI)
    
    mat = Matrix(lI,tI,image_coor)
    
    return mat

   

### Output funtion

In [12]:
def Output_initilize(list1):
    first_half = list1[:len(list1) //2] 
    second_half = list1[len(list1) //2:]
    for i in range(len(first_half)):
        if(first_half[i] == 0):
            my_Label = Label(text="Off", width=10, bg = "red")
            my_Label.grid(row = i+1, pady = 10) 
            global watercount 
            watercount+= 1 
        if(first_half[i] == 1):
            my_Label2 = Label(text="On", width=10, bg = "green") 
            my_Label2.grid(row = i+1, pady = 10) 
    for j in range(len(second_half)):
        if(second_half[j] == 0):
            my_Label3 = Label(text="Off", width=10, bg = "red")
            my_Label3.grid(row = j+1, column = 1, pady = 10)  
            watercount += 1 
        if(second_half[j] == 1):
            my_Label4 = Label(text="On", width=10, bg = "green")
            my_Label4.grid(row = j+1, column = 1, pady = 10) 
      
    Formula = watercount
    strform = str(Formula)
    my_Label5 = Label(text = "                                    Water Saved (gallons): " + strform)
    my_Label5.grid(row = 16, pady = 10) 

## Main 

In [13]:
watercount = 0
def main():
    
    #initialize video (Will be replaced with streaming)
    #video = "Videos/cow_video.mov"
    video = "Videos/cv.mov"

    if not os.path.isfile(video):
        print("File not found!")
    
    
    #Initialize Tkinter
    root = Tk()
    root.title("Cow Detection")
    
    
    #InitilizeVideoinfo
    seconds_frequency = 10
    capture,total_frame,fps,multiplier = VideoInfoExtractor(video,seconds_frequency)
    print(capture)
    print(total_frame)
    print(fps)
    
    frame_counter = 1
    InitialSetup = True
    
    UI_delay = 500
    
    
    while frame_counter <= total_frame: 
        frame = VideotoFrames(capture,frame_counter)
        #frame = imageCrop(frame)
        
        if InitialSetup == True:
            boundingbox_widget = BoundingBoxWidget(frame)
            setup(boundingbox_widget)
            InitialSetup = False
            
        AWS_Matrix = CallAWS(frame,frame_counter)
        
        root.after(UI_delay, Output_initilize , AWS_Matrix)
        UI_delay += 2000
        frame_counter += multiplier
    
    root.mainloop()

    return 0

In [20]:
main()

<VideoCapture 0000019A7A3D7AB0>
2910
30.0
Instructions:
Mark the stanchions with Xs
Press Q to Exit Window
top left: (777, 347), bottom right: (904, 409)


NoCredentialsError: Unable to locate credentials

In [ ]:
#!pip3 install awscli --upgrade --user
